1. Generate definition and didyouknow csv
2. Classify ImageNEt/Dogs images by confidence intervals
3. Classify Adversarial images by confidence intervals and verify attacks
4. Compute E/M/H distributions
5. Determine the best threshold on the original distributions of ImageNet/Stanford Dogs
6. Apply adversarial attacks by Foolbox3
7. Copy input images for expplanation visualizations
8. Generate samples images and intro images for Dogs and ImageNet
9. Generate explanations for training/validation for all tested method

In [3]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision

from torchvision.models import *
from visualisation.core.utils import device, image_net_postprocessing, image_net_preprocessing
from torch import nn
from operator import itemgetter
from visualisation.core.utils import imshow

import xml.etree.ElementTree as ET
import random
import os
from torchvision import models
from shutil import copyfile, rmtree
import glob
import matplotlib.pyplot as plt
import numpy as np
from utils import *
from PIL import Image
from IPython.core.debugger import Tracer

plt.rcParams["figure.figsize"]= 16,8


from torchvision.transforms import ToTensor, Resize, Compose, ToPILImage
from visualisation.core import *

size= 224

# Pre-process the image and convert into a tensor
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
def make_dir(path):
    if os.path.isdir(path) == False:
        os.mkdir(path)

# Added for loading ImageNet classes
def load_imagenet_label_map():
    """
    Load ImageNet label dictionary.
    return:
    """

    input_f = open("input_txt_files/imagenet_classes.txt")
    label_map = {}
    for line in input_f:
        parts = line.strip().split(": ")
        (num, label) = (int(parts[0]), parts[1].replace('"', ""))
        label_map[num] = label

    input_f.close()
    return label_map


# Added for loading ImageNet classes
def load_imagenet_id_map():
    """
    Load ImageNet ID dictionary.
    return;
    """

    input_f = open("input_txt_files/synset_words.txt")
    label_map = {}
    for line in input_f:
        parts = line.strip().split(" ")
        (num, label) = (parts[0], ' '.join(parts[1:]))
        label_map[num] = label

    input_f.close()
    return label_map

def load_imagenet_validation_gt():
    count = 0
    input_f = open("input_txt_files/ILSVRC2012_validation_ground_truth.txt")
    gt_dict = {}
    while True:
        count += 1
        
        # Get the next line
        line = input_f.readline()
        
        # if line is empty, EOL is reached
        if not line:
            break
        gt_dict [count] = int(line.strip())

    input_f.close()
    return gt_dict

def convert_imagenet_label_to_id(label_map, key_list, val_list, prediction_class):
    """
    Convert imagenet label to ID: for example - 245 -> "French bulldog" -> n02108915
    :param label_map:
    :param key_list:
    :param val_list:
    :param prediction_class:
    :return:
    """
    class_to_label = label_map[prediction_class]
    prediction_id = key_list[val_list.index(class_to_label)]
    return prediction_id

gt_dict = load_imagenet_validation_gt()
id_map = load_imagenet_id_map()
label_map = load_imagenet_label_map()

key_list = list(id_map.keys())
val_list = list(id_map.values())

print(key_list[200])

def convert_imagenet_id_to_label(key_list, class_id):
    """
    Convert imagenet label to ID: for example - n02108915 -> "French bulldog" -> 245
    :param label_map:
    :param key_list:
    :param val_list:
    :param prediction_class:
    :return:
    """
    return key_list.index(str(class_id))

n02097474


In [3]:
# Generate csv file for class definitions

import requests
from bs4 import BeautifulSoup
import csv

input_f = open("input_txt_files/gloss.txt")
gloss_dict = dict()

for line in input_f:
    parts = line.strip().split("\t")
    key = parts[0]
    definition = parts[1]
    gloss_dict[key] = definition

definition_dict = dict()
for wnid in key_list:
    definition_dict[wnid] = gloss_dict[wnid].split(';')[0]

with open('csv_files/definition.csv', 'w') as file:
    for key in definition_dict.keys():
        file.write("%s \t %s\n" %(key, definition_dict[key]))

In [ ]:
# Generate "Did you know?" questions
didyouknow_dict = dict()
for key, value in definition_dict.items():
    predicted_label = id_map[key].split(',')[0]
    predicted_label = predicted_label[0].upper() + predicted_label[1:]
    question = "Did you know what **{}** is?".format(predicted_label)
    didyouknow_dict[key] = question
print(didyouknow_dict)
with open('csv_files/didyouknow.csv', 'w') as file:
    for key in didyouknow_dict.keys():
        file.write("%s \t %s\n" %(key, didyouknow_dict[key]))

In [14]:
# Get IDs of 120 dog breeds

def load_imagenet_dog_label():
    count = 0
    dog_id_list = list()
    input_f = open("input_txt_files/dog_type.txt")
    for line in input_f:
        dog_id = (line.split('-')[0])
        dog_id_list.append(dog_id)
    return dog_id_list

dogs_id = load_imagenet_dog_label()


In [15]:
############# Prepare images for all experiments
UPPER_THRESH = 0.8
LOWER_THRESH = 0.2

NAT_DATASET_INTERVAL = '/home/dexter/Downloads/Dataset/Interval_nat/'
DOG_DATASET_INTERVAL = '/home/dexter/Downloads/Dataset/Interval_dog/'
ADV_NAT_INTERVAL = '/home/dexter/Downloads/Dataset/Interval_adv_nat/'
ADV_DOG_INTERVAL = '/home/dexter/Downloads/Dataset/Interval_adv_dog/'

NAT_RESNET34_CORRECT = '/home/dexter/Downloads/Dataset/Natural_correct_resnet34/'
NAT_RESNET34_WRONG = '/home/dexter/Downloads/Dataset/Natural_wrong_resnet34/'
NAT_RESNET34_HARD = '/home/dexter/Downloads/Dataset/Natural_hard_images_resnet34/'
NAT_RESNET34_EASY = '/home/dexter/Downloads/Dataset/Natural_easy_images_resnet34/'
NAT_ADV_RESNET34 = '/home/dexter/Downloads/Dataset/Natural_Adv_images_resnet34/'
NAT_RESNET34_CORRECT_UPPER_THRESH = '/home/dexter/Downloads/Dataset/Natural_correct_resnet34_above_08/'
NAT_RESNET34_CORRECT_LOWER_THRESH = '/home/dexter/Downloads/Dataset/Natural_correct_resnet34_below_02/'
NAT_RESNET34_WRONG_UPPER_THRESH = '/home/dexter/Downloads/Dataset/Natural_wrong_resnet34_above_08/'
NAT_RESNET34_WRONG_LOWER_THRESH = '/home/dexter/Downloads/Dataset/Natural_wrong_resnet34_below_02/'
NAT_ATTACKED = '/home/dexter/Downloads/Dataset/Adv_nat_examples/'
NAT_ATTACKED_VERIFIED = '/home/dexter/Downloads/Dataset/Adv_nat_examples_verified/'

DOG_RESNET34_CORRECT = '/home/dexter/Downloads/Dataset/Dog_correct_resnet34/'
DOG_RESNET34_WRONG = '/home/dexter/Downloads/Dataset/Dog_wrong_resnet34/'
DOG_RESNET34_HARD = '/home/dexter/Downloads/Dataset/Dog_hard_images_resnet34/'
DOG_RESNET34_EASY = '/home/dexter/Downloads/Dataset/Dog_easy_images_resnet34/'
DOG_ADV_RESNET34 = '/home/dexter/Downloads/Dataset/Dog_Adv_images_resnet34/'
DOG_RESNET34_CORRECT_UPPER_THRESH = '/home/dexter/Downloads/Dataset/Dog_correct_resnet34_above_08/'
DOG_RESNET34_CORRECT_LOWER_THRESH = '/home/dexter/Downloads/Dataset/Dog_correct_resnet34_below_02/'
DOG_RESNET34_WRONG_UPPER_THRESH = '/home/dexter/Downloads/Dataset/Dog_wrong_resnet34_above_08/'
DOG_RESNET34_WRONG_LOWER_THRESH = '/home/dexter/Downloads/Dataset/Dog_wrong_resnet34_below_02/'
DOG_ATTACKED = '/home/dexter/Downloads/Dataset/Adv_dog_examples/'
DOG_ATTACKED_VERIFIED = '/home/dexter/Downloads/Dataset/Adv_dog_examples_verified/'

In [ ]:
# Visualize W and H distribution of dataset

print(len(size_dict))
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
w = [value[0] for value in size_dict.values()]
h = [value[1] for value in size_dict.values()]
plt.hist(w, density=True, bins=30)  # `density=False` would make counts
plt.axis([0, 600, 0, 0.005]) 
plt.title('Width distribution')
plt.ylabel('Probability')
plt.xlabel('Data')
plt.figure(figsize=(16,8), dpi=300)
plt.show()
plt.close()

plt.hist(h, density=True, bins=30)  # `density=False` would make counts
plt.axis([0, 600, 0, 0.005]) 
plt.title('Height distribution')
plt.ylabel('Probability')
plt.xlabel('Data')
plt.figure(figsize=(16,8), dpi=300)
plt.show()
plt.close()

In [ ]:
# This block is to categorize correctly/incorrectly classified ADVERSARIAL images and VERIFY ATTACKS
# Remove MINDS
model = resnet34(pretrained=True).to(device)
model.eval()

dog_flag = True

# make_dir(DOG_ATTACKED_VERIFIED)

images = glob.glob(DOG_RESNET34_WRONG + '*.*') 
correct = 0
wrong = 0
indomain_wrong = 0

for idx, image in enumerate(images):
#     print(idx)
    gt = image.split('val_')[1][9:18]
#     gt = image[0:9]
#     print(gt)
    img_name = image.split('resnet34/')[1]
    img = Image.open(image)
    if img.mode != 'RGB':
        continue
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    _, index = torch.topk(p, 1) # Get 1 most probable classes
    category_id = index[0][0].item()
    img_name = img_name.replace("xxx", str(category_id))
    prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, category_id)
    
    if prediction_id == gt:
        correct += 1
    else:
        wrong += 1
        if dog_flag:
            if prediction_id not in dogs_id:
                print(image)
                print(prediction_id)
                continue
            else:
                indomain_wrong += 1
        dst_file = DOG_ATTACKED_VERIFIED + img_name
#         copyfile(image, dst_file)

print([correct, wrong])
print(indomain_wrong)

In [ ]:
print(dogs_id)

In [ ]:
# This block is to categorize correctly/incorrectly classified images by confidence intervals
dog_flag = False
adv_flag = False

model = resnet34(pretrained=True).to(device)
model.eval()

from shutil import copyfile
        
if adv_flag:
    if dog_flag:     
        image_paths = glob.glob(DOG_ATTACKED_VERIFIED + '/*.*')
        dataset_path = ADV_DOG_INTERVAL
    else:
        image_paths = glob.glob(NAT_ATTACKED_VERIFIED + '/*.*')
        dataset_path = ADV_NAT_INTERVAL
#     print(image_paths)
    make_dir(dataset_path)
    correct = 0
    wrong = 0
    indomain_wrong = 0
    for idx, image_path in enumerate(image_paths):
#         Tracer()()
        print(idx)
#         print(image_path)
        imagenet_id = image_path.split('val_')[1][9:18] # To get synset id nxxxx
#         print(image_path)
#         print(imagenet_id)
        img = Image.open(image_path)
        if img.mode != 'RGB' or img.size[0] < 224 or img.size[1] < 224:
            continue
        x = transform(img).unsqueeze(0).to(device)
        out = model(x)
        p = torch.nn.functional.softmax(out, dim=1)
        score, index = torch.topk(p, 1)
        confidence_score = score[0][0].item()
        category_id = int(index[0][0].item())
        prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, category_id)

        conf_interval = str(int(confidence_score*10))
        if int(confidence_score*10) == 10:
            conf_interval = str(9)
        print([prediction_id, imagenet_id])
        if prediction_id == imagenet_id:
            correct +=1
            make_dir(dataset_path + 'correct_' + conf_interval + '/')
            dst_file = dataset_path + 'correct_' + conf_interval + '/' + prediction_id + '_' + image_path.split('verified/')[1]
        else:
            wrong +=1
            if dog_flag:
                if prediction_id not in dogs_id:
                    continue
                else:
                    indomain_wrong += 1
            make_dir(dataset_path + 'wrong_' + conf_interval + '/')
            print(image_path)
            dst_file = dataset_path + 'wrong_' + conf_interval + '/' + prediction_id + '_' + image_path.split('verified/')[1]
#         print(dst_file)
        copyfile(image_path, dst_file)
    print([correct, wrong])
    print(indomain_wrong)
            
else:
    imagenet_folders = glob.glob('/home/dexter/Downloads/val/*')
    if dog_flag:
        dataset_path = DOG_DATASET_INTERVAL
    else:
        dataset_path = NAT_DATASET_INTERVAL
    make_dir(dataset_path)
    correct = 0
    wrong = 0
    indomain_wrong = 0
    for i, imagenet_folder in enumerate(imagenet_folders):
        if i == 100:
            break
        
        imagenet_id = imagenet_folder.split('val/')[1]

        if dog_flag:
            if imagenet_id not in dogs_id:
                continue

        image_paths = glob.glob(imagenet_folder + '/*.*')
        for idx, image_path in enumerate(image_paths):
            print(idx)
            img = Image.open(image_path)
            if img.mode != 'RGB' or img.size[0] < 224 or img.size[1] < 224:
                continue
            x = transform(img).unsqueeze(0).to(device)
            out = model(x)
            p = torch.nn.functional.softmax(out, dim=1)
            score, index = torch.topk(p, 1)
            confidence_score = score[0][0].item()
            category_id = int(index[0][0].item())
            prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, category_id)

            conf_interval = str(int(confidence_score*10))
            if int(confidence_score*10) == 10:
                conf_interval = str(9)
#             print(prediction_id, imagenet_id)
            if prediction_id == imagenet_id:
                correct +=1
                if os.path.isdir(dataset_path + 'correct_' + conf_interval + '/') == False:
                    os.mkdir(dataset_path + 'correct_' + conf_interval + '/')
                dst_file = dataset_path + 'correct_' + conf_interval + '/' + prediction_id + '_' + image_path.split(imagenet_id + '/')[1].split('.JPEG')[0] + '_' + imagenet_id + '.jpeg'
            else:
                wrong +=1
                # Ensure the prediction is in dog ids
                if dog_flag:
                    if prediction_id not in dogs_id:
                        continue
                    else:
                        indomain_wrong += 1
                if os.path.isdir(dataset_path + 'wrong_' + conf_interval + '/') == False:
                    os.mkdir(dataset_path + 'wrong_' + conf_interval + '/')
                dst_file = dataset_path + 'wrong_' + conf_interval + '/' + prediction_id + '_' + image_path.split(imagenet_id + '/')[1].split('.JPEG')[0] + '_' + imagenet_id + '.jpeg'
            copyfile(image_path, dst_file)


    print([correct, wrong])
    print(indomain_wrong)

In [ ]:
# This block is to get the hard/easy/normal distribution of ImageNet/Stanford Dogs
dog_flag = True

model = resnet34(pretrained=True).to(device)
model.eval()
hard = 0
easy = 0
normal = 0

from shutil import copyfile
        
imagenet_folders = glob.glob('/home/dexter/Downloads/val/*')

if dog_flag:
    dataset_path = DOG_DATASET_INTERVAL
else:
    dataset_path = NAT_DATASET_INTERVAL
    
make_dir(dataset_path)
correct = 0
wrong = 0
indomain_wrong = 0
for i, imagenet_folder in enumerate(imagenet_folders):
    print(i)
#     if i == 10:
#         break
    imagenet_id = imagenet_folder.split('val/')[1]

    if dog_flag:
        if imagenet_id not in dogs_id:
            continue

    image_paths = glob.glob(imagenet_folder + '/*.*')
    for idx, image_path in enumerate(image_paths):
#         print(idx)
        img = Image.open(image_path)
        if img.mode != 'RGB' or img.size[0] < 224 or img.size[1] < 224:
            continue
        x = transform(img).unsqueeze(0).to(device)
        out = model(x)
        p = torch.nn.functional.softmax(out, dim=1)
        score, index = torch.topk(p, 1)
        confidence_score = score[0][0].item()
        category_id = int(index[0][0].item())
        prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, category_id)

        conf_interval = str(int(confidence_score*10))
        if int(confidence_score*10) == 10:
            conf_interval = str(9)
#         print(prediction_id, imagenet_id)
        if prediction_id == imagenet_id:
            correct +=1
        else:
            wrong +=1
            # Ensure the prediction is in dog ids
            if dog_flag:
                if prediction_id not in dogs_id:
                    continue
                else:
                    indomain_wrong += 1
        # Normal
        conf_interval = int(conf_interval)
        if conf_interval in [4,5]:
            normal += 1
        # Easy
        elif (conf_interval in [0,1,2] and prediction_id != imagenet_id) or (conf_interval in [8,9] and prediction_id == imagenet_id):
            easy += 1
        # Hard
        elif (conf_interval in [0,1,2] and prediction_id == imagenet_id) or (conf_interval in [8,9] and prediction_id != imagenet_id):
            hard += 1

print(normal,easy,hard)
print([correct, wrong])
print(indomain_wrong)

In [8]:
# This block is to determine the best threshold on the original distributions of ImageNet/Stanford Dogs
T_list = list(np.arange(0.0,1.0,0.05))
for T in T_list:
    # This block is to get the hard/easy/normal distribution of ImageNet/Stanford Dogs
    dog_flag = False

    model = resnet34(pretrained=True).to(device)
    model.eval()
    correct = 0
    wrong = 0

    imagenet_folders = glob.glob('/home/dexter/Downloads/val/*')

    indomain_wrong = 0
    for i, imagenet_folder in enumerate(imagenet_folders):
    #    print(i)
    #     if i == 10:
    #         break
        imagenet_id = imagenet_folder.split('val/')[1]

        if dog_flag:
            if imagenet_id not in dogs_id:
                continue

        image_paths = glob.glob(imagenet_folder + '/*.*')
        for idx, image_path in enumerate(image_paths):
    #         print(idx)
            img = Image.open(image_path)
            if img.mode != 'RGB' or img.size[0] < 224 or img.size[1] < 224:
                continue
            x = transform(img).unsqueeze(0).to(device)
            out = model(x)
            p = torch.nn.functional.softmax(out, dim=1)
            score, index = torch.topk(p, 1)
            confidence_score = score[0][0].item()
            category_id = int(index[0][0].item())
            prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, category_id)
            
            conf_interval = str(int(confidence_score*10))
            conf_interval = int(conf_interval)
            if conf_interval in [3,6,7]:
                continue

            if (confidence_score > T and prediction_id == imagenet_id) or (confidence_score <= T and prediction_id != imagenet_id):
                correct +=1
            else:
                wrong +=1
                # Ensure the prediction is in dog ids
                if dog_flag:
                    if prediction_id not in dogs_id:
                        continue
                    else:
                        indomain_wrong += 1
                        
    print("Accuracy at T = {} is {}".format(T, correct*100/(correct+wrong)))


Accuracy at T = 0.0 is 77.65231160605353
Accuracy at T = 0.05 is 77.66009916153986
Accuracy at T = 0.1 is 78.07543545414428
Accuracy at T = 0.15000000000000002 is 78.99177114970277
Accuracy at T = 0.2 is 80.24037587934481
Accuracy at T = 0.25 is 81.73299068089193
Accuracy at T = 0.30000000000000004 is 83.10360044648651
Accuracy at T = 0.35000000000000003 is 83.10360044648651
Accuracy at T = 0.4 is 83.10360044648651
Accuracy at T = 0.45 is 83.94465643901046
Accuracy at T = 0.5 is 84.5079562858552
Accuracy at T = 0.55 is 84.80128754250707
Accuracy at T = 0.6000000000000001 is 84.54170235962931
Accuracy at T = 0.65 is 84.54170235962931
Accuracy at T = 0.7000000000000001 is 84.54170235962931
Accuracy at T = 0.75 is 84.54170235962931
Accuracy at T = 0.8 is 84.54170235962931
Accuracy at T = 0.8500000000000001 is 81.80567453209771
Accuracy at T = 0.9 is 77.46541027438154
Accuracy at T = 0.9500000000000001 is 70.18404589466033


In [ ]:
# Apply adversarial attacks by Foolbox

# FOOLBOX 3
# pip install foolbox==3.2.1
import foolbox
# import matplotlib.pyplot as plt

from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, FGSM

# instantiate a model (could also be a TensorFlow or JAX model)
model = models.resnet34(pretrained=True).eval()
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

# Pre-process the image and convert into a tensor
transform_t = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
#     torchvision.transforms.CenterCrop(256),
    torchvision.transforms.ToTensor()
])

make_dir(DOG_ATTACKED)

attack = foolbox.attacks.PGD()
epsilons = [16/255]

test_image_paths = glob.glob(DOG_RESNET34_CORRECT + '*.*')
print(len(test_image_paths))

# test_image_paths = test_image_paths[:100]
test_images = list()
test_inputs = list()
for test_image_path in test_image_paths:
    img = Image.open(test_image_path)
    test_inputs.append(transform_t(img).unsqueeze(0))
    img.close()

batch_num = int(len(test_inputs)/100)
print(batch_num)
for idx in range(batch_num):
    print(idx)
    # Apply adversarial attacks on each 100 images/increase the size if you have better memory
    labels = [x.split('.jpeg')[0][-9:] for x in test_image_paths[idx*100:idx*100+100]]
    img_names = [(x.split('resnet34/')[1]).split('.jpeg')[0] for x in test_image_paths[idx*100:idx*100+100]]
    labels = [convert_imagenet_id_to_label(key_list, x) for x in labels]
    labels = np.array(labels)
    test_input = np.concatenate(test_inputs[idx*100:idx*100+100])
    images= test_input
   
    images = torch.from_numpy(images).to(device)
    labels = torch.from_numpy(labels).to(device)
    _, adversarials, _ = attack(fmodel, images, labels, epsilons=epsilons)

    for i in range(len(adversarials[0])):
#         perturbation_size = ((adversarials[0][i] - images[i]), (adversarials[0][i] - images[i]))
        adversarial = adversarials[0][i]
        visual = adversarial.cpu().numpy()
        rescaled = (255.0 / visual.max() * (visual - visual.min())).astype(np.uint8)
        rescaled = np.swapaxes(np.swapaxes(rescaled, 1,0), 2,1)

        im = Image.fromarray(rescaled)
    
        orig_path = test_image_paths[idx*100+i]
        pert_path = '{}{}_{}_{}.jpeg'.format(DOG_ATTACKED, img_names[i], labels[i].item(), 'xxx')

        im.save(pert_path, bbox_inches='tight')
print("Done!")

In [ ]:
# Copy input image for explanation visualization
# import random

exp_num = 4 # NAT/Dog/Adv
exps = ["Natural", "Dog", "Adversarial_Nat", "Adversarial_Dog"]
DATASET_INTERVAL = [NAT_DATASET_INTERVAL, DOG_DATASET_INTERVAL, ADV_NAT_INTERVAL, ADV_DOG_INTERVAL]
# NAT
# 1st step: Get img_per_bin from len(bins)
for idx, exp in enumerate(exps):
    print(exp)
#     EXP1 = '/home/dexter/Downloads/New_experiments/{}/AIonly/'.format(exp)
    EXP1 = '/home/dexter/Downloads/Human_experiments/Dataset/{}/'.format(exp)
    
#     if exp == "Dog" or exp == "Adversarial_Dog":
    if 1:
        img_per_bin = 50 # 7 methods
        bins = {'bin1':[0,1,2], 'bin2':[4,5], 'bin3':[8,9]}
#     else:
#         img_per_bin = 35
#         bins = {'bin1':[0,1], 'bin2':[4,5], 'bin3':[8,9]}
    
    make_dir(EXP1)
    make_dir(EXP1 + 'mixed_images/')
    make_dir(EXP1 + 'correct_images/')
    make_dir(EXP1 + 'wrong_images/')
    for bin_name in list(bins.keys()):
        make_dir(EXP1 + 'correct' + '_' + bin_name + '_images/')
        make_dir(EXP1 + 'wrong' + '_' + bin_name + '_images/')

        bin_int = bins.get(bin_name)
        print(bin_int)
        correct_images = []
        for intv in bin_int:
            correct_images += glob.glob(DATASET_INTERVAL[idx] + 'correct_' + str(intv) + '/*.*')
#             correct_image_paths = correct_images[:img_per_bin]
        if exp in ["Natural", "Dog"]:
            correct_image_paths = random.sample(correct_images, img_per_bin)
            for correct_image_path in correct_image_paths:
                copyfile(correct_image_path, EXP1 + 'correct' + '_' + bin_name + '_images/' + correct_image_path.split('correct_')[1][2:])
                copyfile(correct_image_path, EXP1 + 'correct_images/' + correct_image_path.split('correct_')[1][2:])
                copyfile(correct_image_path, EXP1 + 'mixed_images/' + correct_image_path.split('correct_')[1][2:])
        
        wrong_images = []
        for intv in bin_int:
            wrong_images += glob.glob(DATASET_INTERVAL[idx] + 'wrong_' + str(intv) + '/*.*')
#             wrong_image_paths = wrong_images[:img_per_bin]
        wrong_image_paths = random.sample(wrong_images, img_per_bin)
        for wrong_image_path in wrong_image_paths:
            if exp in ["Natural", "Dog"]:
                copyfile(wrong_image_path, EXP1 + 'wrong' + '_' + bin_name + '_images/' + wrong_image_path.split('wrong_')[1][2:])
                copyfile(wrong_image_path, EXP1 + 'wrong_images/' + wrong_image_path.split('wrong_')[1][2:])
                copyfile(wrong_image_path, EXP1 + 'mixed_images/' + wrong_image_path.split('wrong_')[1][2:])
            else:
                copyfile(wrong_image_path, EXP1 + 'wrong' + '_' + bin_name + '_images/' + wrong_image_path.split('wrong_')[1][2:])
                copyfile(wrong_image_path, EXP1 + 'wrong_images/' + wrong_image_path.split('wrong_')[1][2:])
                copyfile(wrong_image_path, EXP1 + 'mixed_images/' + wrong_image_path.split('wrong_')[1][2:])
    

In [ ]:
# Dataset Analysis
cd /home/dexter/Downloads/Dataset
sh run.sh > analysis.txt

In [ ]:
# Generate intro images for natural image classification
from shutil import copyfile
imagenet_folders = glob.glob('/home/dexter/Downloads/train/*')

correct = 0
wrong = 0
cmd = 'montage '
SPACE = ' '
for idx, imagenet_folder in enumerate(imagenet_folders):
    if idx == 50:
        break
    imagenet_id = imagenet_folder.split('train/')[1]
    print(imagenet_id)
    image_paths = glob.glob(imagenet_folder + '/*.*')
    print(image_paths[0])
    predicted_label = id_map[imagenet_id].split(',')[0]
    print(id_map[imagenet_id])
    predicted_label = predicted_label[0].lower() + predicted_label[1:]
    predicted_label = predicted_label.replace(' ','_')
    print(predicted_label)
    cmd += SPACE + '-label' + SPACE + predicted_label + SPACE + image_paths[10]
    
cmd += SPACE + '-tile 10x5 -title \'1000 classes\' tmp/natural_image_classification.jpeg'
os.system(cmd)
    

In [ ]:
# Generate intro images for dog image classification
from shutil import copyfile
imagenet_folders = glob.glob('/home/dexter/Downloads/train/*')

cnt = 0
correct = 0
wrong = 0
cmd = 'montage '
SPACE = ' '
for idx, imagenet_folder in enumerate(imagenet_folders):
    if cnt == 50:
        break
    imagenet_id = imagenet_folder.split('train/')[1]
    if imagenet_id not in dogs_id:
        continue
    cnt += 1
    print(imagenet_id)
    image_paths = glob.glob(imagenet_folder + '/*.*')
    print(image_paths[0])
    predicted_label = id_map[imagenet_id].split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]
    predicted_label = predicted_label.replace(' ','_')
    print(predicted_label)
    cmd += SPACE + '-label' + SPACE + predicted_label + SPACE + image_paths[1]
    
cmd += SPACE + '-tile 10x5 -title \'120 classes\' tmp/dog_image_classification.jpeg'
os.system(cmd)


In [13]:
model = resnet34(pretrained=True).to(device)
model.eval()
# Generate sample images for 1000 ImageNet classes with high-confidence images
from shutil import copyfile
imagenet_folders = glob.glob('/home/dexter/Downloads/train/*')
dog_flag = False
samples_folder = 'sample_images/'
SPACE = ' '
import cv2

for idx, imagenet_folder in enumerate(imagenet_folders):
    print(idx)
    imagenet_id = imagenet_folder.split('train/')[1]
    #if imagenet_id != 'n02085620':
    #    continue
    if dog_flag:
        samples_folder = 'sample_images_dog/'
        if imagenet_id not in dogs_id:
            continue
    image_paths = glob.glob(imagenet_folder + '/*.*')
    random_image_paths = list()
    for i, image_path in enumerate(image_paths):
        if len(random_image_paths) == 3:
            break
        print(i)
        img = Image.open(image_path)
        if img.mode != 'RGB':
            continue
        x = transform(img).unsqueeze(0).to(device)
        out = model(x)
        p = torch.nn.functional.softmax(out, dim=1)
        score, index = torch.topk(p, 1)
        confidence_score = score[0][0].item()
        category_id = int(index[0][0].item())
        input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, category_id)
        # Selected based on confidence score
        if confidence_score > 0.9 and input_prediction_id == imagenet_id:
            print(confidence_score, category_id)
            random_image_paths.append(image_path)
    if len(random_image_paths) < 3:
        random_image_paths = random.sample(image_paths, 3)
        
    while cv2.imread(random_image_paths[0]).shape[0] < 320 or cv2.imread(random_image_paths[1]).shape[0] < 320 or cv2.imread(random_image_paths[2]).shape[0] < 320 or \
            cv2.imread(random_image_paths[0]).shape[1] < 240 or cv2.imread(random_image_paths[1]).shape[1] < 240 or cv2.imread(random_image_paths[2]).shape[1] < 240:
        random_image_paths = random.sample(list(image_paths), 3)
        
    predicted_label = id_map[imagenet_id].split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]
    predicted_label = predicted_label.replace(' ','_')
    cmd = 'montage -mode concatenate' + SPACE + random_image_paths[0] + SPACE + random_image_paths[1] + SPACE + random_image_paths[2] + \
        SPACE + '-resize 320x240 -pointsize 8 -geometry +2+30' + SPACE + samples_folder + \
        imagenet_id + '.jpeg'
    cmd = 'convert {} -resize 600x600\! tmp/s0.jpeg'.format(random_image_paths[0])
    os.system(cmd)
    cmd = 'convert {} -resize 600x600\! tmp/s1.jpeg'.format(random_image_paths[1])
    os.system(cmd)
    cmd = 'convert {} -resize 600x600\! tmp/s2.jpeg'.format(random_image_paths[2])
    os.system(cmd)
    
    myCmd = 'montage tmp/s[0-2].jpeg -tile 3x1 -geometry +0+0 {}{}.jpeg'.format(samples_folder, imagenet_id)
    os.system(myCmd)

0
0
1
2
3
4
5
6
7
8
9
10
11
0.9543769359588623 998
12
0.9628803730010986 998
13
14
15
0.9088001251220703 998
1
0
1
2
0.9774549007415771 337
3
0.9271901249885559 337
4
0.9816599488258362 337
2
0
0.9999328851699829 540
1
0.9910323023796082 540
2
0.9993891716003418 540
3
0
0.9229702353477478 299
1
2
3
0.9254594445228577 299
4
5
0.9851928949356079 299
4
0
0.9914906620979309 774
1
2
3
4
5
0.9239859580993652 774
6
7
0.9365164041519165 774
5
0
0.999970555305481 45
1
0.9997370839118958 45
2
0.9998238682746887 45
6
0
0.967336893081665 965
1
2
0.9836795926094055 965
3
0.9881844520568848 965
7
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129


17
18
19
0.9001078009605408 893
75
0
0.9490306377410889 499
1
0.9895660877227783 499
2
0.999994158744812 499
76
0
1
0.9019393920898438 264
2
3
4
5
6
7
0.9224335551261902 264
8
0.9973726272583008 264
77
0
1
0.9958503246307373 442
2
0.9810246229171753 442
3
0.9979804158210754 442
78
0
0.9928898811340332 659
1
2
0.9831386804580688 659
3
4
0.9985729455947876 659
79
0
0.999963641166687 556
1
0.9999856948852539 556
2
0.939403235912323 556
80
0
0.9960888624191284 924
1
0.992344081401825 924
2
3
4
0.9486720561981201 924
81
0
0.9858958125114441 562
1
2
3
4
0.9919314980506897 562
5
6
0.9817200303077698 562
82
0
1
2
3
0.962124764919281 718
4
5
0.9034006595611572 718
6
7
8
9
10
11
12
13
14
15
0.9815497994422913 718
83
0
0.9995083808898926 793
1
0.9825394153594971 793
2
0.997214138507843 793
84
0
1
0.9938575625419617 259
2
0.9702659845352173 259
3
0.9479816555976868 259
85
0
1
2
3
4
0.987825334072113 400
5
6
7
8
9
10
11
0.9731566905975342 400
12
0.9161748886108398 400
86
0
1
0.9942225813865662 224


744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993


526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
0.927709698677063 68
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770

286
287
288
289
290
291
292
293
294
295
296
0.9129468202590942 837
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
0.9951013922691345 837
212
0
1
2
0.9989508390426636 109
3
0.998727023601532 109
4
0.9992770552635193 109
213
0
1
0.9481765627861023 897
2
0.9983707070350647 897
3
0.9909111857414246 897
214
0
0.9999576807022095 533
1
0.9980828762054443 533
2
0.9975417852401733 533
215
0
0.980466902256012 37
1
2
3
4
0.9307183027267456 37
5
6
7
8
9
10
0.9936762452125549 37
216
0
0.9758338332176208 366
1
2
0.9866622090339661 366
3
0.9982106685638428 366
217
0
0.9999996

...

698
0
0.9932236671447754 735
1
2
3
0.9044674038887024 735
4
5
0.9994321465492249 735
699
0
1
0.9985778331756592 934
2
3
4
0.9595797657966614 934
5
0.9998600482940674 934
700
0
1
2
0.9488834738731384 254
3
0.9535946249961853 254
4
0.9848929643630981 254
701
0
0.9999967813491821 583
1
2
0.9868148565292358 583
3
0.9867308139801025 583
702
0
1
2
0.9997261166572571 452
3
0.9922312498092651 452
4
5
0.9501475691795349 452
703
0
1
2
3
4
5
6
7
8
0.9383571743965149 238
9
10
11
12
0.9549521207809448 238
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0.9769802093505859 238
704
0
1
0.9998648166656494 806
2
0.999487042427063 806
3
4
0.9764556884765625 806
705
0
0.9969172477722168 327
1
0.9935048818588257 327
2
0.9548700451850891 327
706
0
0.9645830988883972 578
1
2
3
4
5
6
7
0.9867854714393616 578
8
9
10
0.9690876603126526 578
707
0
0.9582439064979553 93
1
0.9643483757972717 93
2
0.9998626708984375 93
708
0
0.9982666969299316 286
1
0.9989204406738281 286
2
0.9263170957565308 286
709
0
0.99

763
0
0.9999414682388306 988
1
0.999572217464447 988
2
3
0.9929649829864502 988
764
0
0.9815953969955444 237
1
0.9897869825363159 237
2
3
0.9331261515617371 237
765
0
0.9998433589935303 105
1
0.9971616268157959 105
2
0.946861743927002 105
766
0
1
0.9974074959754944 673
2
3
0.9997860789299011 673
4
5
6
0.9962863922119141 673
767
0
0.9998403787612915 51
1
0.9965319633483887 51
2
0.9476385116577148 51
768
0
0.9999682903289795 145
1
0.999663233757019 145
2
0.9997830986976624 145
769
0
1
2
0.9980970025062561 905
3
0.9954248070716858 905
4
0.9979173541069031 905
770
0
0.998898983001709 455
1
0.95991051197052 455
2
0.9999986886978149 455
771
0
0.991266667842865 173
1
0.9945138692855835 173
2
0.9904058575630188 173
772
0
1
2
3
4
5
0.973271906375885 446
6
7
8
0.9933804273605347 446
9
10
0.9932250380516052 446
773
0
0.9999016523361206 414
1
0.9871668219566345 414
2
3
0.9985994696617126 414
774
0
0.995233952999115 794
1
0.9921699166297913 794
2
0.9997604489326477 794
775
0
1
2
0.9999819993972778 

820
0
0.9687527418136597 484
1
0.9446153044700623 484
2
3
4
5
6
0.9682236313819885 484
821
0
1
0.9984649419784546 335
2
0.9560976028442383 335
3
4
5
0.9997630715370178 335
822
0
1
0.9787592887878418 127
2
0.9689456224441528 127
3
0.9791740775108337 127
823
0
1
0.9992263317108154 491
2
3
4
0.9978452920913696 491
5
0.9950644373893738 491
824
0
0.9093701839447021 268
1
0.9957420229911804 268
2
0.9995934367179871 268
825
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0.9413678646087646 689
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
0.9002274870872498 689
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160

894
0
1
2
3
4
0.9803786277770996 702
5
6
0.9054526686668396 702
7
0.9901637434959412 702
895
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
0.987711489200592 311
23
0.9546837210655212 311
24
25
26
27
28
0.933853805065155 311
896
0
0.9991262555122375 389
1
2
0.9970300197601318 389
3
0.9750223755836487 389
897
0
1
0.9998347759246826 521
2
3
0.9994487166404724 521
4
0.9401566386222839 521
898
0
1
2
3
4
5
0.9720486998558044 211
6
0.9879686832427979 211
7
0.9995298385620117 211
899
0
0.9977561831474304 387
1
0.9999760389328003 387
2
0.9996780157089233 387
900
0
0.9748563766479492 687
1
0.9999581575393677 687
2
0.9953851103782654 687
901
0
1
2
3
4
5
0.9998767375946045 341
6
0.9911137223243713 341
7
0.9316105246543884 341
902
0
1
2
0.9886186122894287 941
3
0.9941330552101135 941
4
0.9852185845375061 941
903
0
0.9990891218185425 133
1
0.9993813037872314 133
2
0.9953368306159973 133
904
0
0.9969890713691711 873
1
0.9286168217658997 873
2
0.9995311498641968 873
905
0
0.95363157987594

991
0
0.9991962313652039 768
1
0.9999029636383057 768
2
0.9963499307632446 768
992
0
1
2
0.9677216410636902 460
3
0.9728773832321167 460
4
0.9280748963356018 460
993
0
1
2
3
0.9623147249221802 970
4
5
6
7
8
9
10
11
12
13
14
15
16
0.9663472175598145 970
17
18
19
20
21
0.9932488203048706 970
994
0
1
2
3
0.9594613313674927 582
4
5
0.9513944983482361 582
6
0.954958975315094 582
995
0
0.9955429434776306 50
1
2
3
4
0.9916191101074219 50
5
0.9979938268661499 50
996
0
0.9973320960998535 666
1
0.949722170829773 666
2
0.9999891519546509 666
997
0
0.9887518286705017 906
1
0.9797697067260742 906
2
3
4
0.9787424802780151 906
998
0
0.9246942400932312 950
1
2
0.9965820908546448 950
3
4
0.9589247107505798 950
999
0
0.9994284510612488 293
1
0.9832119345664978 293
2
3
0.9744720458984375 293


In [ ]:
model = resnet34(pretrained=True).to(device)
model.eval()
import cv2 as cv
method = 'Conf'
# test_image_paths = corrects_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num]
# test_image_paths = ['/home/dexter/Downloads/Dataset/Interval_train_nat/correct_0/n03250847_n03250847_8348_n03250847.jpeg']

test_image_paths = glob.glob('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/Inputs/No/*.*') + \
                    glob.glob('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/Inputs/Yes/*.*')
# for idx, image_path in enumerate(test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]):
for idx, image_path in enumerate(test_image_paths):
    print(idx + 1)
    distance_dict = dict()
    neighbors = list()
    categories_confidences = list()
    confidences= list ()

    img = Image.open(image_path)

    input_image = img.resize((size,size), Image.ANTIALIAS)
    if '/No/' in image_path:
        image_name = (image_path.split('.jpeg')[0]).split('No/')[1]
    else:
        image_name = (image_path.split('.jpeg')[0]).split('Yes/')[1]
    print(image_name)

    # Get the prediction for the input image
    img = Image.open(image_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)


    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]

    # Original image
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}: {}'.format(predicted_label, predicted_confidence))
    plt.imshow(input_image)
#     plt.imshow(modified_img[idx])
    plt.savefig('tmp/original.jpeg', bbox_inches='tight')
    plt.close()


    img = cv.resize(cv.imread(image_path,0),((size,size)))
    edges = cv.Canny(img,100,200)
    edges = edges - 255

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.imshow(edges, cmap = 'gray')
    plt.savefig('tmp/Edge.jpeg', bbox_inches='tight')
    plt.close()

#     Confidence score only
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}% {}'.format(int(float(predicted_confidence)*100), predicted_label))
    plt.imshow(input_image)
    plt.savefig('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/{}/{}_{}.jpeg'.format(method, image_name, int(float(predicted_confidence)*100)), bbox_inches='tight', dpi=300)
    plt.close()


In [ ]:
model = resnet34(pretrained=True).to(device)
model.eval()
from torchray.attribution.grad_cam import grad_cam
method = 'GradCAM'
## Creating colormap
from matplotlib import cm
cMap = 'Reds'
import cv2 as cv
test_image_paths = glob.glob('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/Inputs/No/*.*') + \
                    glob.glob('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/Inputs/Yes/*.*')

# for idx, image_path in enumerate(test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]):
for idx, image_path in enumerate(test_image_paths):
    print(idx + 1)
    distance_dict = dict()
    neighbors = list()
    categories_confidences = list()
    confidences= list ()
    img = Image.open(image_path)
#     print(test_image_paths[idx])
    input_image = img.resize((size,size), Image.ANTIALIAS)
    input_image = img.resize((size,size), Image.ANTIALIAS)
    if '/No/' in image_path:
        image_name = (image_path.split('.jpeg')[0]).split('No/')[1]
    else:
        image_name = (image_path.split('.jpeg')[0]).split('Yes/')[1]
    print(image_name)

    # Get the prediction for the input image
    img = Image.open(image_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)
#     print(predicted_confidence)


    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]

    # Original image
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}% {}'.format(int(float(predicted_confidence)*100), predicted_label))
    plt.imshow(input_image)
#     plt.imshow(modified_img[idx])
    plt.savefig('tmp/original.jpeg', bbox_inches='tight')
    plt.close()

    img = cv.resize(cv.imread(image_path,0),((size,size)))
    edges = cv.Canny(img,100,200)
    edges = edges - 255

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.imshow(edges, cmap = 'gray')
    plt.savefig('tmp/Edge.jpeg', bbox_inches='tight')
    plt.close()
    

#     GRAD-CAM
    saliency = grad_cam(
        model, x, input_category_id,
        saliency_layer='layer4',
        resize=True
    )

    saliency_path = 'saliency_maps/GradCAM_resnet34/'
    saliency *= 1.0/saliency.max()
    GradCAM = saliency[0][0].cpu().detach().numpy()

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('Explanation')
    plt.imshow(GradCAM, cmap=cMap, vmin=0, vmax=1)
    plt.colorbar(orientation='vertical')
    plt.savefig('tmp/heatmap.jpeg', bbox_inches='tight')
    plt.close()

    # Get overlay version
    myCmd = 'composite -blend 10 tmp/Edge.jpeg -gravity SouthWest tmp/heatmap.jpeg tmp/overlay.jpeg'
    os.system(myCmd)

    img_path = '/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/{}/{}_{}.jpeg'.format(method, image_name, int(float(predicted_confidence)*100))
    myCmd = 'montage tmp/original.jpeg tmp/overlay.jpeg -tile 2x1 -geometry +0+0 ' + 'tmp/tmp.jpeg'
    print(myCmd)
    os.system(myCmd)
    copyfile('tmp/tmp.jpeg', img_path)

In [ ]:
model = resnet34(pretrained=True).to(device)
model.eval()
imagenet_train_path = '/home/dexter/Downloads/train'
K = 3
layer = 4
feature_extractor = nn.Sequential(*list(model.children())[:layer-6])
method = 'NNs'
# NNs
import cv2 as cv

test_image_paths = glob.glob('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/Inputs/No/*.*') + \
                    glob.glob('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/Inputs/Yes/*.*')

# for idx, image_path in enumerate(test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]):
for idx, image_path in enumerate(test_image_paths):
    print(idx + 1)
    distance_dict = dict()
    neighbors = list()
    categories_confidences = list()
    confidences= list ()
    img = Image.open(image_path)
    
    if 1:
        embedding = feature_extractor(transform(img).unsqueeze(0).to(device)).flatten(start_dim=1) 
#     print(test_image_paths[idx])
    input_image = img.resize((size,size), Image.ANTIALIAS)
    if '/No/' in image_path:
        image_name = (image_path.split('.jpeg')[0]).split('No/')[1]
    else:
        image_name = (image_path.split('.jpeg')[0]).split('Yes/')[1]
    print(image_name)


    # Get the prediction for the input image
    img = Image.open(image_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)

    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]
    
    if os.path.exists('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/{}/{}_{}.jpeg'.format(method, image_name, int(float(predicted_confidence)*100))) == True:
        print("Continue")
        continue

    # Original image
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}% {}'.format(int(float(predicted_confidence)*100), predicted_label))
    plt.imshow(input_image)
#     plt.imshow(modified_img[idx])
    plt.savefig('tmp/original.jpeg', bbox_inches='tight')
    plt.close()

    img = cv.resize(cv.imread(image_path,0),((size,size)))
    edges = cv.Canny(img,100,200)
    edges = edges - 255

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.imshow(edges, cmap = 'gray')
    plt.savefig('tmp/Edge.jpeg', bbox_inches='tight')
    plt.close()
    
    if 1:
        from utils import *
        ## Nearest Neighbors

        predicted_set_path = os.path.join(imagenet_train_path, input_prediction_id)
        predicted_set_img_paths = glob.glob(predicted_set_path + '/*.*')
        predicted_set_color_images= list()

        embedding = embedding.detach()
        embedding.to(device)
        # Build search space for nearest neighbors
        for i, path in enumerate(predicted_set_img_paths):
            img = Image.open(path)
            if img.mode != 'RGB':
                img.close()
                del img
                continue

            x = transform(img).unsqueeze(0).to(device)
            out = model(x)
            p = torch.nn.functional.softmax(out, dim=1)
            del out
            score, index = torch.topk(p, 1)
            del p
            category_id = index[0][0].item()
            del score, index
            
            # This is to avoid the confusion from crane 134 and crane 517 and to make NNs work :)
            if input_category_id != 134 and input_category_id != 517 and category_id != 134 and category_id != 517:
                if input_category_id != category_id:
                    continue

            f = feature_extractor(x)
            feature_vector = f.flatten(start_dim=1).to(device)
            feature_vector = feature_vector.detach()

            del f
            distance_dict[path] = torch.dist(embedding, feature_vector)

            del feature_vector 
            torch.cuda.empty_cache()
            img.close()
            del img
            predicted_set_color_images.append(path)

        # Get K most similar images
        res = dict(sorted(distance_dict.items(), key = itemgetter(1))[:K]) 
        print("Before...")
        print(res)
        while distance_dict[list(res.keys())[0]] < 100:
            del distance_dict[list(res.keys())[0]]
            res = dict(sorted(distance_dict.items(), key = itemgetter(1))[:K]) 
        print("After...")
        print(res)
#         del distance_dict
        del embedding

        similar_images = list(res.keys())
#         print(similar_images)
#         print([distance_dict[x] for x in similar_images])

        for similar_image in similar_images:
            img = Image.open(similar_image)
            neighbors.append(img.resize((size,size), Image.ANTIALIAS))
            x = transform(img).unsqueeze(0).to(device)
            out = model(x)
            p = torch.nn.functional.softmax(out, dim=1)
            score, index = torch.topk(p, 1) # Get 1 most probable classes
            category_id = index[0][0].item()
            confidence = score[0][0].item()

            label = label_map.get(category_id).split(',')[0].replace("\"", "")
            label = label[0].lower() + label[1:]
            print(label + ": %.2f" %(confidence))

            categories_confidences.append((label + ": %.2f" %(confidence)))
            confidences.append(confidence)
            img.close()
            
        img_path = '/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/{}/{}_{}.jpeg'.format(method, image_name, int(float(predicted_confidence)*100))
        print(img_path)
        font = {'color': 'red',
               'weight': 'bold',
               }
        for index, neighbor in enumerate(neighbors):
            fig = plt.figure()
            plt.figure(figsize=(6.0,4.5), dpi=300)
            plt.axis('off')
            if index == 1: # Make title for the middle image (2nd image) to annotate the 3 NNs
                plt.title('Three examples of {}'.format(predicted_label), fontdict=font)
            else:
                plt.title(' ')
            plt.imshow(neighbor)
            plt.savefig('tmp/{}.jpeg'.format(index), bbox_inches='tight')
            plt.close()
            
        myCmd = 'montage tmp/[0-2].jpeg -tile 3x1 -geometry +0+0 ' + 'tmp/aggregate.jpeg'
        os.system(myCmd)

        myCmd = 'montage tmp/original.jpeg ' + 'tmp/aggregate.jpeg' + ' -tile 2x -geometry +120+0 ' + 'tmp/tmp.jpeg'
        os.system(myCmd)
        copyfile('tmp/tmp.jpeg', img_path)

In [ ]:
from torchray.attribution.extremal_perturbation import extremal_perturbation, contrastive_reward
from torchray.attribution.grad_cam import grad_cam
import PIL.Image

cMap = 'Reds'


def get_EP_saliency_maps(model, path):
        img_index = (path.split('.jpeg')[0]).split('Inputs/')[1]
        img = PIL.Image.open(path)
        x = transform(img).unsqueeze(0).to(device)
        out = model(x)
        p = torch.nn.functional.softmax(out, dim=1)
        score, index = torch.topk(p, 1)
        category_id_1 = index[0][0].item()

        masks, energy = extremal_perturbation(
            model, x, category_id_1,
            areas=[0.025, 0.05, 0.1, 0.2],
            num_levels=8,
            step=7,
            sigma=7 * 3,
            max_iter=800,
            debug=False,
            jitter=True,
            smooth=0.09,
            perturbation='blur'
        )
        saliency = masks.sum(dim=0, keepdim=True)
        saliency = saliency.detach()
        saliency_t = saliency.cpu().detach().numpy()[0, 0, :]

        return (saliency[0].to('cpu')), (masks[0].to('cpu')), (masks[1].to('cpu')), (masks[2].to('cpu')) , (masks[3].to('cpu'))

In [ ]:
model = resnet34(pretrained=True).to(device)
model.eval()
method = 'EP'
import cv2 as cv
test_image_paths = glob.glob('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/Inputs/No/*.*') + \
                    glob.glob('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/Inputs/Yes/*.*')
# test_image_paths = corrects_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin1[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin2[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     corrects_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num] + \
#                     wrongs_bin3[methods.index(method)*img_num:methods.index(method)*img_num + img_num]
# for idx, image_path in enumerate(test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]):
for idx, image_path in enumerate(test_image_paths):
    print(idx + 1)
    distance_dict = dict()
    neighbors = list()
    categories_confidences = list()
    confidences= list ()
    img = Image.open(image_path)

    input_image = img.resize((size,size), Image.ANTIALIAS)

    if '/No/' in image_path:
        image_name = (image_path.split('.jpeg')[0]).split('No/')[1]
    else:
        image_name = (image_path.split('.jpeg')[0]).split('Yes/')[1]
    print(image_name)

    # Get the prediction for the input image
    img = Image.open(image_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)
#     print(predicted_confidence)


    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]

    # Original image
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}% {}'.format(int(float(predicted_confidence)*100), predicted_label))
    plt.imshow(input_image)
#     plt.imshow(modified_img[idx])
    plt.savefig('tmp/original.jpeg', bbox_inches='tight')
    plt.close()

    img = cv.resize(cv.imread(image_path,0),((size,size)))
    edges = cv.Canny(img,100,200)
    edges = edges - 255

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.imshow(edges, cmap = 'gray')
    plt.savefig('tmp/Edge.jpeg', bbox_inches='tight')
    plt.close()
    

    # Extremal Perturbation
    saliency,m0,m1,m2,m3 = get_EP_saliency_maps(model, image_path)
    ep_saliency_map = tensor2img(saliency)
    ep_saliency_map *= 1.0/ep_saliency_map.max()

#     ep_saliency_map = np.mean(ep_saliency_map, axis=-1)

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('Explanation')
    plt.imshow(ep_saliency_map, cmap=cMap, vmin=0, vmax=1)
    plt.colorbar(orientation='vertical')
    plt.savefig('tmp/heatmap.jpeg', bbox_inches='tight')
    plt.close()

    # Get overlay version
    myCmd = 'composite -blend 10 tmp/Edge.jpeg -gravity SouthWest tmp/heatmap.jpeg tmp/overlay.jpeg'
    os.system(myCmd)

    img_path = '/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/{}/{}_{}.jpeg'.format(method, image_name, int(float(predicted_confidence)*100))
    myCmd = 'montage tmp/original.jpeg tmp/overlay.jpeg -tile 2x1 -geometry +0+0 tmp/tmp.jpeg'
    os.system(myCmd)
        
    copyfile('tmp/tmp.jpeg', img_path)
#----------------------------------------------------------------------------------------------------

In [ ]:
method = 'PoolNet'
from shutil import copyfile, rmtree
def rm_and_mkdir(path):
    if os.path.isdir(path) == True:
        rmtree(path)
    os.mkdir(path)
    
cMap = 'Reds'

# Prepare dataset
rm_and_mkdir('/home/dexter/Downloads/run-0/run-0-sal-p/')
rm_and_mkdir('/home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs/')
if os.path.isdir('/home/dexter/Downloads/PoolNet-master/data/PASCALS/test.lst'):
    os.remove('/home/dexter/Downloads/PoolNet-master/data/PASCALS/test.lst')

test_image_paths = glob.glob('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/Inputs/No/*.*') + \
                    glob.glob('/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/Inputs/Yes/*.*')

# src_paths = test_image_paths[methods.index(method)*img_num:methods.index(method)*img_num + img_num]
src_paths = test_image_paths
for src_path in src_paths:
    if '/No/' in src_path:
        dst_path = '/home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs/' + src_path.split('No/')[1]
    else:
        dst_path = '/home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs/' + src_path.split('Yes/')[1]
    copyfile(src_path, dst_path)

cmd = 'ls /home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs/ > /home/dexter/Downloads/PoolNet-master/data/PASCALS/test.lst'
os.system(cmd)
cmd = 'python /home/dexter/Downloads/PoolNet-master/main.py --mode=\'test\' --model=\'/home/dexter/Downloads/run-0/run-0/models/final.pth\' --test_fold=\'/home/dexter/Downloads/run-0/run-0-sal-p/\' --sal_mode=\'p\''
os.system(cmd)


npy_file_paths = glob.glob('/home/dexter/Downloads/run-0/run-0-sal-p/*.*')
print(len(npy_file_paths))
for idx, npy_file_path in enumerate(npy_file_paths): 
    print(idx)
    test_path = '/home/dexter/Downloads/PoolNet-master/data/PASCALS/Imgs' + npy_file_path.split('.npy')[0].split('sal-p')[1]
    image_name = npy_file_path.split('.npy')[0].split('sal-p')[1]
    img = Image.open(test_path)
    input_image = img.resize((size,size), Image.ANTIALIAS)
    
    # Retrieve confidence and label
    img = Image.open(test_path)
    x = transform(img).unsqueeze(0).to(device)
    out = model(x)
    p = torch.nn.functional.softmax(out, dim=1)
    score, index = torch.topk(p, 1)
    input_category_id = index[0][0].item()
    predicted_confidence = score[0][0].item()
    predicted_confidence = ("%.2f") %(predicted_confidence)
    
    # For overlay heatmap
    img = cv.resize(cv.imread(test_path,0),((size,size)))
    edges = cv.Canny(img,100,200)
    edges = edges - 255

    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.imshow(edges, cmap = 'gray')
    plt.savefig('tmp/Edge.jpeg', bbox_inches='tight')
    plt.close()
    ##############################

    
    input_prediction_id = convert_imagenet_label_to_id(label_map, key_list, val_list, input_category_id)
    predicted_label = id_map.get(input_prediction_id).split(',')[0]
    predicted_label = predicted_label[0].lower() + predicted_label[1:]
    
    # Plot input image
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('{}% {}'.format(int(float(predicted_confidence)*100), predicted_label))
    plt.imshow(input_image)
    plt.savefig('tmp/original.jpeg', bbox_inches='tight')
    
    # Plot the explanation
    npy_file = np.load(npy_file_path)
    fig = plt.figure()
    plt.figure(figsize=(6.0,4.5), dpi=300)
    plt.axis('off')
    plt.title('Explanation')
    plt.imshow(npy_file, cmap=cMap, vmin=0, vmax=1)
    plt.colorbar(orientation='vertical')
    plt.savefig('tmp/heatmap.jpeg', bbox_inches='tight')
    
    # Get overlay version
    myCmd = 'composite -blend 10 Edge.jpeg -gravity SouthWest heatmap.jpeg overlay.jpeg'
    os.system(myCmd)

    image_name = image_name.split('.jpeg')[0]
    img_path = '/home/dexter/Downloads/Human_experiments/Stimuli_Dog/Validation/{}/{}_{}.jpeg'.format(method, image_name, int(float(predicted_confidence)*100))
    myCmd = 'montage tmp/original.jpeg tmp/overlay.jpeg -tile 2x1 -geometry +0+0 tmp/tmp.jpeg'
    os.system(myCmd)
        
    copyfile('tmp/tmp.jpeg', img_path)
    